In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load stock market data
nyse_df_2019=pd.read_csv('data/NYA_2019.csv')
nyse_df_2019.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,11238.769531,11407.799805,11204.280273,11383.530273,11383.530273,3733160000
1,2019-01-03,11343.790039,11343.790039,11169.459961,11190.440430,11190.440430,3822860000
2,2019-01-04,11323.730469,11559.019531,11323.730469,11533.339844,11533.339844,4213410000
3,2019-01-07,11536.049805,11678.969727,11504.769531,11605.959961,11605.959961,4104710000
4,2019-01-08,11702.500000,11727.750000,11621.150391,11716.230469,11716.230469,4081030000


In [3]:
nyse_df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 7 columns):
Date         251 non-null object
Open         251 non-null float64
High         251 non-null float64
Low          251 non-null float64
Close        251 non-null float64
Adj Close    251 non-null float64
Volume       251 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 13.8+ KB


In [4]:
#Load Weather data
# IV. FORMAT OF "ghcnd-stations.txt"
#------------------------------
#Variable   Columns   Type
#------------------------------
#ID            1-11   Character
#LATITUDE     13-20   Real
#LONGITUDE    22-30   Real
#ELEVATION    32-37   Real
#STATE        39-40   Character
#NAME         42-71   Character
#GSN FLAG     73-75   Character
#HCN/CRN FLAG 77-79   Character
#WMO ID       81-85   Character
#------------------------------

In [5]:
stations_file = open('data/aws-data/ghcnd-stations.txt', 'r') 
lines = stations_file.readlines()

newyork_stations =  []

for line in lines: 
    line = line.strip()
    newyork_stations.append((line[:11], line[12:20], line[21:30],line[38:40]))
    
newyork_stations_df = pd.DataFrame(newyork_stations)
newyork_stations_df = newyork_stations_df[newyork_stations_df[3] == 'NY']
newyork_stations_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1594 entries, 75552 to 114237
Data columns (total 4 columns):
0    1594 non-null object
1    1594 non-null object
2    1594 non-null object
3    1594 non-null object
dtypes: object(4)
memory usage: 62.3+ KB


In [6]:
newyork_stations_df.head()

,0,1,2,3
75552,US1NYAB0001,42.6670,-74.0509,NY
75553,US1NYAB0006,42.7198,-73.9304,NY
75554,US1NYAB0010,42.5455,-74.1475,NY
75555,US1NYAB0016,42.7590,-73.7370,NY
75556,US1NYAB0017,42.6678,-73.7856,NY


In [7]:
import geopandas
from shapely.geometry import Point, Polygon

nybb_path = geopandas.datasets.get_path('nybb')

boros = geopandas.read_file(nybb_path)
boros = boros.to_crs(epsg=4326)
boros

,BoroCode,BoroName,Shape_Leng,Shape_Area,geometry
0,5,Staten Island,330470.010332,1.623820e+09,(POLYGON ((-74.05050806403247 40.5664220341607...
1,4,Queens,896344.047763,3.045213e+09,(POLYGON ((-73.83668274106707 40.5949466970158...
2,3,Brooklyn,741080.523166,1.937479e+09,(POLYGON ((-73.86706149472118 40.5820879767934...
3,1,Manhattan,359299.096471,6.364715e+08,(POLYGON ((-74.01092841268031 40.6844914725429...
4,2,Bronx,464392.991824,1.186925e+09,(POLYGON ((-73.89680883223774 40.7958084451597...


In [8]:
def isInNewYorkCity(lon, lat):
    point = Point(float(lat), float(lon))
    for boroPloy in boros['geometry']:
        if boroPloy.contains(point):
            return True
    return False


isNewYorks = []    
for i, row in newyork_stations_df.iterrows():
    isNewYorks.append(isInNewYorkCity(row[1],row[2]))
    
newyork_stations_df['isInNYC']= isNewYorks
newyork_stations_df = newyork_stations_df[newyork_stations_df['isInNYC'] == True]
nyc_stations_df = newyork_stations_df.drop([1,2,3,'isInNYC'], axis=1)
nyc_stations_df.head()

,0
75594,US1NYBX0025
75841,US1NYKN0003
75842,US1NYKN0025
75937,US1NYNY0074
76048,US1NYQN0002


In [9]:
import numpy as np

def getWeatherInfo(file):
    weather_df = pd.read_csv(file, header=None)
    nyc_weather_df = pd.merge(weather_df, nyc_stations_df, on=[0, 0])
    nyc_weather_df = nyc_weather_df.drop([4,5,6,7],axis=1)
    nyc_weather_df.columns = ['id', 'Date', 'type', 'value']
    nyc_weather_df = nyc_weather_df[(nyc_weather_df['type'] == 'PRCP') | (nyc_weather_df['type'] == 'SNOW') | (nyc_weather_df['type'] == 'SNWD') 
                    | (nyc_weather_df['type'] == 'AWDR') | (nyc_weather_df['type'] == 'AWND')]
    nyc_weather_df = pd.pivot_table(nyc_weather_df, values='value', index=['Date'], columns=['type'], aggfunc=np.mean)
    nyc_weather_df = nyc_weather_df.reset_index()
    nyc_weather_df['Date'] = nyc_weather_df.Date.apply(str)
    nyc_weather_df['Date'] = nyc_weather_df.Date.apply(lambda x: '{}-{}-{}'.format(x[:4], x[4:6], x[6:8]))
    return nyc_weather_df

nyc_weather_df_2019 = getWeatherInfo('data/aws-data/2019.csv')
nyc_weather_df_2019.head()

type,Date,AWND,PRCP,SNOW,SNWD
0,2019-01-01,79.0,233.400000,0.0,0.0
1,2019-01-02,37.5,0.000000,0.0,0.0
2,2019-01-03,49.0,0.333333,0.0,0.0
3,2019-01-04,48.0,0.000000,0.0,0.0
4,2019-01-05,57.0,91.125000,0.0,0.0


In [10]:
def getStockAndWeatherDataFrame(stock_df, weather_df):
    nyc_stocl_weather_df = pd.merge(stock_df, weather_df, on=['Date', 'Date'])
    return nyc_stocl_weather_df

nyc_stock_weather_df_2019 = getStockAndWeatherDataFrame(nyse_df_2019, nyc_weather_df_2019)
nyc_stock_weather_df_2019.head()

,Date,Open,High,Low,Close,Adj Close,Volume,AWND,PRCP,SNOW,SNWD
0,2019-01-02,11238.769531,11407.799805,11204.280273,11383.530273,11383.530273,3733160000,37.5,0.000000,0.0,0.0
1,2019-01-03,11343.790039,11343.790039,11169.459961,11190.440430,11190.440430,3822860000,49.0,0.333333,0.0,0.0
2,2019-01-04,11323.730469,11559.019531,11323.730469,11533.339844,11533.339844,4213410000,48.0,0.000000,0.0,0.0
3,2019-01-07,11536.049805,11678.969727,11504.769531,11605.959961,11605.959961,4104710000,45.5,0.000000,0.0,0.0
4,2019-01-08,11702.500000,11727.750000,11621.150391,11716.230469,11716.230469,4081030000,30.0,29.125000,0.0,0.0


In [11]:
nyc_stock_weather_df_2019.drop(['Adj Close'], inplace=True, axis=1)
nyc_stock_weather_df_2019.head()

,Date,Open,High,Low,Close,Volume,AWND,PRCP,SNOW,SNWD
0,2019-01-02,11238.769531,11407.799805,11204.280273,11383.530273,3733160000,37.5,0.000000,0.0,0.0
1,2019-01-03,11343.790039,11343.790039,11169.459961,11190.440430,3822860000,49.0,0.333333,0.0,0.0
2,2019-01-04,11323.730469,11559.019531,11323.730469,11533.339844,4213410000,48.0,0.000000,0.0,0.0
3,2019-01-07,11536.049805,11678.969727,11504.769531,11605.959961,4104710000,45.5,0.000000,0.0,0.0
4,2019-01-08,11702.500000,11727.750000,11621.150391,11716.230469,4081030000,30.0,29.125000,0.0,0.0


In [12]:
SNOW_HIGH_TH = 25.0
SNOW_MEDIUM_TH = 15.0
nyc_stock_weather_df_2019['SNOW'] = nyc_stock_weather_df_2019.SNOW.apply(lambda x : 0 if x == 0.0 else 1)
nyc_stock_weather_df_2019['SNOW_Intensity'] = nyc_stock_weather_df_2019.SNWD.apply(lambda x: 2 if x > SNOW_HIGH_TH else ( 1 if x > SNOW_MEDIUM_TH else 0))

PRCP_HIGH_TH = 30.0
PRCP_MEDIUM_TH = 20.0
nyc_stock_weather_df_2019['RAIN'] = nyc_stock_weather_df_2019.PRCP.apply(lambda x : 0 if x == 0.0 else 1)
nyc_stock_weather_df_2019['RAIN_Intensity'] = nyc_stock_weather_df_2019.PRCP.apply(lambda x: 2 if x > PRCP_HIGH_TH else ( 1 if x > PRCP_MEDIUM_TH else 0))

AWND_HIGH_TH = 30.0
AWND_MEDIUM_TH = 20.0
nyc_stock_weather_df_2019['WINDY'] = nyc_stock_weather_df_2019.AWND.apply(lambda x: 2 if x > AWND_HIGH_TH else ( 1 if x > AWND_MEDIUM_TH else 0))
nyc_stock_weather_df_2019.drop(['AWND','PRCP','SNWD'], inplace=True, axis=1)
nyc_stock_weather_df_2019.head()

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

In [16]:
features = ['Open', 'Low','Close', 'Volume', 'SNOW','SNOW_Intensity','RAIN','RAIN_Intensity','WINDY']

# Separating out the features
X = nyc_stock_weather_df_2019.loc[:, features].values
# Separating out the target
y = nyc_stock_weather_df_2019.loc[:,['Open']].values
# Standardizing the features
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [17]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import NearestNeighbors
import keras
from keras.models import Sequential
from keras.layers import Dense

linear_reg = LinearRegression()
dt_reg = DecisionTreeRegressor(criterion='mse', max_depth=3)    
svr_linear = SVR(kernel='linear', C=100, gamma='auto')
linear_ridge = Ridge()

# Define model
ml_model = Sequential()
ml_model.add(Dense(500, input_dim=9, activation= "relu"))
ml_model.add(Dense(100, activation= "relu"))
ml_model.add(Dense(50, activation= "relu"))
ml_model.add(Dense(1))
ml_model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

linear_reg.fit(X_train, y_train)
dt_reg.fit(X_train, y_train)
svr_linear.fit(X_train, y_train)
linear_ridge.fit(X_train, y_train)
ml_model.fit(X_train, y_train, epochs=20)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
175/175 [==============================] - 1s 5ms/step - loss: 164977342.0800 - mean_squared_error: 164977344.0000
Epoch 2/20
175/175 [==============================] - 0s 278us/step - loss: 164920142.0800 - mean_squared_error: 164920144.0000
Epoch 3/20
175/175 [==============================] - 0s 285us/step - loss: 164819369.8743 - mean_squared_error: 164819376.0000
Epoch 4/20
175/175 [==============================] - 0s 263us/step - loss: 164652070.0343 - mean_squared_error: 164652064.0000
Epoch 5/20
175/175 [==============================] - 0s 286us/step - loss: 164361430.6743 - mean_squared_error: 164361440.0000
Epoch 6/20
175/175 [==============================] - 0s 244us/step - loss: 163899100.3429 - mean_squared_error: 163899104.0000
Epoch 7/20
175/175 [==============================] - 0s 290us/step - loss: 163161828.2971 - mean_squared_error: 163161824.0000
Epoch 8/20
175/175 [==============================] - 0s 2

In [19]:

models =['linear_regession', 'linear_ridge', 'decision_tree_reg', 'svr_linear', 'ann_model']

for model_name, model in zip(models,  [linear_reg, linear_ridge, dt_reg, svr_linear, ml_model]):
    y_pred = model.predict(X_test)
    print("{}  mse:{}".format(model_name, mean_squared_error(y_test,y_pred)))
    print("{}  rmse:{}".format(model_name, np.sqrt(mean_squared_error(y_test,y_pred))))
    

linear_regession  mse:0.0
linear_regession  rmse:0.0
linear_ridge  mse:198.2666525408622
linear_ridge  rmse:14.080719176976089
decision_tree_reg  mse:7666.430694209642
decision_tree_reg  rmse:87.55815606903586
svr_linear  mse:0.007080794860337416
svr_linear  rmse:0.08414745902484172
ann_model  mse:63238939.52506186
ann_model  rmse:7952.291463789658


<p>As decision tree and ann model has higer as mse on test so there are likely Overfitting. on the other hand linear regression and svr_linear mse is zero wich likey to be underfitting. </p>

<p>
Detecting overfitting is usefull but it doesnot solved the problem. Here are serveral other option 
we can do like 
<li>Cross Validation </li>
<li> Training on more data </li>
<li> Remove features </li>
<li> Early stoping </li>
<li> Regularization </li>
<li> Ensembling </li>

In this dataset we can increase trainset and Regularizaion to improve model performance 
</p>

<h4> Two Years of data this time </h4>

In [21]:
nyc_weather_df_2018 = getWeatherInfo('data/aws-data/2018.csv')
nyc_weather_df_2018.head()

type,Date,AWND,PRCP,SNOW,SNWD
0,2018-01-01,63.333333,0.0,0.000000,7.60
1,2018-01-02,56.000000,0.0,0.000000,1.25
2,2018-01-03,27.333333,0.0,0.000000,0.00
3,2018-01-04,97.666667,95.4,135.000000,27.00
4,2018-01-05,93.666667,113.5,122.166667,203.60


In [22]:
#load stock market data
nyse_df_2018=pd.read_csv('data/NYA_2018.csv')
nyse_df_2018.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-02,12860.929688,12902.719727,12842.269531,12902.719727,12902.719727,3357250000
1,2018-01-03,12910.629883,12963.389648,12906.990234,12957.280273,12957.280273,3538660000
2,2018-01-04,12999.530273,13045.160156,12999.530273,13028.459961,13028.459961,3695260000
3,2018-01-05,13066.370117,13104.519531,13039.080078,13103.230469,13103.230469,3236620000
4,2018-01-08,13096.650391,13117.240234,13065.240234,13114.349609,13114.349609,3242650000


In [25]:
nyc_stock_weather_df_2018 = getStockAndWeatherDataFrame(nyse_df_2018, nyc_weather_df_2018)
nyc_stock_weather_df_2018.drop(['Adj Close'], inplace=True, axis=1)
nyc_stock_weather_df_2018.head()

,Date,Open,High,Low,Close,Volume,AWND,PRCP,SNOW,SNWD
0,2018-01-02,12860.929688,12902.719727,12842.269531,12902.719727,3357250000,56.000000,0.000000,0.000000,1.25
1,2018-01-03,12910.629883,12963.389648,12906.990234,12957.280273,3538660000,27.333333,0.000000,0.000000,0.00
2,2018-01-04,12999.530273,13045.160156,12999.530273,13028.459961,3695260000,97.666667,95.400000,135.000000,27.00
3,2018-01-05,13066.370117,13104.519531,13039.080078,13103.230469,3236620000,93.666667,113.500000,122.166667,203.60
4,2018-01-08,13096.650391,13117.240234,13065.240234,13114.349609,3242650000,45.666667,1.142857,0.000000,159.25


In [27]:
SNOW_HIGH_TH = 25.0
SNOW_MEDIUM_TH = 15.0
nyc_stock_weather_df_2018['SNOW'] = nyc_stock_weather_df_2018.SNOW.apply(lambda x : 0 if x == 0.0 else 1)
nyc_stock_weather_df_2018['SNOW_Intensity'] = nyc_stock_weather_df_2018.SNWD.apply(lambda x: 2 if x > SNOW_HIGH_TH else ( 1 if x > SNOW_MEDIUM_TH else 0))

PRCP_HIGH_TH = 30.0
PRCP_MEDIUM_TH = 20.0
nyc_stock_weather_df_2018['RAIN'] = nyc_stock_weather_df_2018.PRCP.apply(lambda x : 0 if x == 0.0 else 1)
nyc_stock_weather_df_2018['RAIN_Intensity'] = nyc_stock_weather_df_2018.PRCP.apply(lambda x: 2 if x > PRCP_HIGH_TH else ( 1 if x > PRCP_MEDIUM_TH else 0))

AWND_HIGH_TH = 30.0
AWND_MEDIUM_TH = 20.0
nyc_stock_weather_df_2018['WINDY'] = nyc_stock_weather_df_2018.AWND.apply(lambda x: 2 if x > AWND_HIGH_TH else ( 1 if x > AWND_MEDIUM_TH else 0))
nyc_stock_weather_df_2018.drop(['AWND','PRCP','SNWD'], inplace=True, axis=1)
nyc_stock_weather_df_2018.head()

,Date,Open,High,Low,Close,Volume,SNOW,SNOW_Intensity,RAIN,RAIN_Intensity,WINDY
0,2018-01-02,12860.929688,12902.719727,12842.269531,12902.719727,3357250000,0,0,0,0,2
1,2018-01-03,12910.629883,12963.389648,12906.990234,12957.280273,3538660000,0,0,0,0,1
2,2018-01-04,12999.530273,13045.160156,12999.530273,13028.459961,3695260000,1,2,1,2,2
3,2018-01-05,13066.370117,13104.519531,13039.080078,13103.230469,3236620000,1,2,1,2,2
4,2018-01-08,13096.650391,13117.240234,13065.240234,13114.349609,3242650000,0,2,1,0,2


In [28]:
nyc_stock_weather_df_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 11 columns):
Date              250 non-null object
Open              250 non-null float64
High              250 non-null float64
Low               250 non-null float64
Close             250 non-null float64
Volume            250 non-null int64
SNOW              250 non-null int64
SNOW_Intensity    250 non-null int64
RAIN              250 non-null int64
RAIN_Intensity    250 non-null int64
WINDY             250 non-null int64
dtypes: float64(4), int64(6), object(1)
memory usage: 23.4+ KB


In [31]:
nyc_stock_weather_df_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 250
Data columns (total 11 columns):
Date              251 non-null object
Open              251 non-null float64
High              251 non-null float64
Low               251 non-null float64
Close             251 non-null float64
Volume            251 non-null int64
SNOW              251 non-null int64
SNOW_Intensity    251 non-null int64
RAIN              251 non-null int64
RAIN_Intensity    251 non-null int64
WINDY             251 non-null int64
dtypes: float64(4), int64(6), object(1)
memory usage: 23.5+ KB


In [32]:
nyc_stock_weather_df_2018_2019 = pd.concat([nyc_stock_weather_df_2018, nyc_stock_weather_df_2019])
nyc_stock_weather_df_2018_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501 entries, 0 to 250
Data columns (total 11 columns):
Date              501 non-null object
Open              501 non-null float64
High              501 non-null float64
Low               501 non-null float64
Close             501 non-null float64
Volume            501 non-null int64
SNOW              501 non-null int64
SNOW_Intensity    501 non-null int64
RAIN              501 non-null int64
RAIN_Intensity    501 non-null int64
WINDY             501 non-null int64
dtypes: float64(4), int64(6), object(1)
memory usage: 47.0+ KB


In [34]:
# Separating out the features
X = nyc_stock_weather_df_2018_2019.loc[:, features].values
# Separating out the target
y = nyc_stock_weather_df_2018_2019.loc[:,['Open']].values
# Standardizing the features
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

linear_reg = LinearRegression()
dt_reg = DecisionTreeRegressor(criterion='mse', max_depth=3)    
svr_linear = SVR(kernel='linear', C=100, gamma='auto')
linear_ridge = Ridge()

# Define model
ml_model = Sequential()
ml_model.add(Dense(500, input_dim=9, activation= "relu"))
ml_model.add(Dense(100, activation= "relu"))
ml_model.add(Dense(50, activation= "relu"))
ml_model.add(Dense(1))
ml_model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])


linear_reg.fit(X_train, y_train)
dt_reg.fit(X_train, y_train)
svr_linear.fit(X_train, y_train)
linear_ridge.fit(X_train, y_train)
ml_model.fit(X_train, y_train, epochs=20)

Epoch 1/20
350/350 [==============================] - 1s 2ms/step - loss: 162654278.5829 - mean_squared_error: 162654272.0000
Epoch 2/20
350/350 [==============================] - 0s 265us/step - loss: 162579073.2800 - mean_squared_error: 162579072.0000
Epoch 3/20
350/350 [==============================] - 0s 232us/step - loss: 162389362.4686 - mean_squared_error: 162389344.0000
Epoch 4/20
350/350 [==============================] - 0s 236us/step - loss: 161938626.8343 - mean_squared_error: 161938608.0000
Epoch 5/20
350/350 [==============================] - 0s 237us/step - loss: 160937334.3086 - mean_squared_error: 160937344.0000
Epoch 6/20
350/350 [==============================] - 0s 246us/step - loss: 159001439.4514 - mean_squared_error: 159001424.0000
Epoch 7/20
350/350 [==============================] - 0s 241us/step - loss: 155491949.3486 - mean_squared_error: 155491936.0000
Epoch 8/20
350/350 [==============================] - 0s 237us/step - loss: 149702316.6171 - mean_squared_

In [35]:
for model_name, model in zip(models,  [linear_reg, linear_ridge, dt_reg, svr_linear, ml_model]):
    y_pred = model.predict(X_test)
    print("{}  mse:{}".format(model_name, mean_squared_error(y_test,y_pred)))
    print("{}  rmse:{}".format(model_name, np.sqrt(mean_squared_error(y_test,y_pred))))

linear_regession  mse:6.573620762010816e-26
linear_regession  rmse:2.563907323210185e-13
linear_ridge  mse:67.92817198681227
linear_ridge  rmse:8.241854887512414
decision_tree_reg  mse:10522.582751572896
decision_tree_reg  rmse:102.57964101893171
svr_linear  mse:0.003282256949584589
svr_linear  rmse:0.057290984889287674
ann_model  mse:10001432.012772357
ann_model  rmse:3162.504073162967


<p>After increasing data data each model's mse imporved but is still has problem of underfitting and overfitting</p>